In [45]:
import pandas as pd
import chardet
import nltk
from nltk.corpus import stopwords
from collections import Counter
import re
from sklearn.feature_extraction.text import CountVectorizer
from twitter_program import connect_Twitter,tweet_clean_tp,tweet_parse_tp,retrieve_hashtags,retrieve_mentions,retrieve_urls,pos_tags,extract_emojis,spell_check,lemmatise
from nltk.stem import PorterStemmer
import numpy as np
import tensorflow as tf

In [2]:
import os
os.getcwd()

'C:\\Users\\Manjusha Pattadkal'

In [4]:
os.chdir(r"C:\Users\Manjusha Pattadkal\Documents\ML Project")

In [49]:
cooccur_dep=pd.read_csv("cooccuring_depressed_updated.csv",encoding='mbcs',header=None)
cooccur_nondep=pd.read_csv("cooccuring_nd_updated.csv",encoding='mbcs',header=None)

In [5]:
usernames = pd.read_csv('usernames_depressed.txt',index_col= False,header=None,sep='\t',error_bad_lines=False)
rowcount = len(usernames.axes[0])

usernames=usernames.values.tolist()

In [6]:
def preprocessing(String): #default value is always true for stemming and stopwords
    
    '''
    This function is used for preprocessing
    - Tokenization
    - Stemming
    - Stop Words
        
    '''
    tokens = nltk.word_tokenize(String)
    token = [word for word in tokens if word.isalpha()]
    influential_words = " ".join(token)
    influential_words = influential_words.lower()
    influential_words = influential_words.split()
    stop_words = set(stopwords.words('english'))
    influentialwords = []
    
    for w in influential_words:
        if w not in stop_words: 
            influentialwords.append(w)
   # stemwords_string = " ".join(influentialwords)
    #words = set(nltk.corpus.words.words())
    #sent = stemwords_string
   # sent = " ".join(w for w in nltk.wordpunct_tokenize(sent) if w.lower() in words)
    
    ps= PorterStemmer()
    
    # if stopwordFlag==False:
    stemwords=[]    
    for w in influentialwords:
          stemwords.append(ps.stem(w))
    stemwords_string = " ".join(stemwords)
    return stemwords_string


In [7]:
def preprocess(data,fun):
    data_df=pd.DataFrame(data)
    pre = data_df[0].apply(fun)
    return pre

In [75]:
usernames=pd.read_csv('usernames_nondepressed.txt',index_col= False,header=None,sep='\t',error_bad_lines=False)

In [83]:
usernames[0][0]

'2solidmiya_'

In [84]:
dep_tweets = []
#dep_vec=[]
#nondep_vec = []
for i in range (0,rowcount):
        dep_tweets = []
    
        filename = filename=r"C:\Users\Manjusha Pattadkal\Documents\ML Project\non depressed\2 months\userdata_"+usernames[0][i]+"_2_months.csv"
    
    
        tweets=pd.read_csv(filename,encoding='mbcs')
        tweets.dropna(subset = ['twittername'],inplace=True)

        row=len(tweets.axes[0])
        
        for i in range (0,row):
            dep_tweets.append(str(tweets.iloc[i,4]))
            
        clean = preprocess(dep_tweets,tweet_clean_tp)
        tags = preprocess(clean,pos_tags)
        nan_value = float("NaN")
        tags.replace("",nan_value, inplace=True)
        tags.dropna(inplace=True)
        tags=tags.reset_index(drop=True)
        rowc = len(tags)
        final_text = []
        for i in range(0,rowc):
            final_text.append(str(preprocessing(tags[i])))
        x = pd.Series(final_text)

        for i in range(len(final_text)):
            tokens=final_text[i].split()
            if len(tokens)==1:
                x.drop(i,inplace=True)

        final_text=x.reset_index(drop=True)
        DF = {}
        for i in range(len(final_text)):
            tokens = final_text[i].split()
            for w in tokens:
                try:
                    DF[w].add(i)
                except:
                    DF[w] = {i}
        token_dict = {}
        for i in range(len(final_text)):
            tokens = final_text[i].split()
            #counter = Counter(tokens + final_text[i])
            for token in tokens:
                #tf = counter[token]/words_count
                if token in token_dict.keys():
                    token_dict[token] = token_dict[token]+1
                else:
                    token_dict[token] = 1
        freq = int((len(final_text)*0.2)/100)
        token_list = []
        count = 0
        for i in token_dict.keys():
             if token_dict[i]>=freq:
                    token_list.append(i)
                    count+=1
        word_list =[]
        new_tweets = []
        for w in token_list:

            for i in range(0,len(final_text)):
                word_list=[]
                if w in final_text[i].split():

                        word_list.append(final_text[i])
                else:
                    continue


                if len(word_list) !=0 and word_list not in new_tweets:
                    new_tweets.append(word_list)
        new_df = pd.DataFrame(new_tweets)
        #new_df.to_csv("new_tweets.csv",index = False, header=False)
        vector_1,a1 = vectorization(new_tweets,new_tweets)
        vector_transpose = np.transpose(vector_1)
        vector_tf=tf.convert_to_tensor(vector_1, dtype=None, dtype_hint=None, name=None)
        vector_t_tf=tf.convert_to_tensor(vector_transpose, dtype=None, dtype_hint=None, name=None)

        result = tf.matmul(vector_t_tf,vector_tf)
        result_arr=result.numpy()
        index_ = []
        for i in range (len(a1)):
             for j in range (0,len(a1)):
                if result_arr[i][j] >0 and j<i:
                    index_.append([result_arr[i][j],i,j])
        list_freq = []
        for i in range (len(index_)):
            list_freq.append(index_[i][0])
        x = np.array(list_freq) 
        f = (max(np.unique(x)) * 10)/100
        index_ = []
        for i in range (len(a1)):
             for j in range (0,len(a1)):
                if result_arr[i][j] >int(f) and j<i:
                    index_.append([result_arr[i][j],i,j])
        co_occur = []
        for i in range (len(index_)):
            a = index_[i][1]
            b = index_[i][2]
            co_occur.append([a1[a],a1[b],index_[i][0]])
        count_dep = 0
        for i in range(0,len(co_occur)):
            a = co_occur[i][0]
            b=co_occur[i][1]
            for j in range(0,len(cooccur_dep)):
                if a == cooccur_dep[0][j] and b == cooccur_dep[1][j]:
                    count_dep = count_dep + 1
        count_ndep = 0
        for i in range(0,len(co_occur)):
            a = co_occur[i][0]
            b=co_occur[i][1]
            for j in range(0,len(cooccur_nondep)):
                if a == cooccur_nondep[0][j] and b == cooccur_nondep[1][j]:
                    count_ndep = count_ndep + 1
        
        dep_vec.append((1865/len(co_occur))*(count_dep/2215))
        nondep_vec.append((500.55/len(co_occur))*(count_ndep/638))
        cat.append(0)
                         
          

        

KeyError: 100

In [93]:
vec_df = pd.DataFrame(zip(dep_vec,nondep_vec,cat))

In [96]:
vec_df.to_csv(r"cooccurences_vector.csv",header=None)

In [76]:
cat = [1] * len(dep_vec)

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [30]:
def doc_freq(token):
    return len(DF[token])

In [31]:
def vectorization(Stringfit,Stringtransform):
    count = []
    vectorizer = CountVectorizer(min_df=freq)
    String_df = pd.DataFrame(Stringfit)
    Stringtransform_df = pd.DataFrame(Stringtransform)
    vectorizer.fit(String_df[0])
    countmatrix = vectorizer.transform(Stringtransform_df[0])
    count=countmatrix.todense()
    return count,vectorizer.get_feature_names()